In [4]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

import requests
from bs4 import BeautifulSoup as bs

from time import sleep
from time import time 

import random

import pandas as pd

In [13]:
class crawler():
    def __init__(self, dname, page):
        '''Initialise class
        Parameter:
        dname: list of district name
        page: integer, representing the number of pages that will scraped for each district'''

        # create a self dictionary to save scraped url of all properties
        self.district_lis = dname
        self.district_url = {i:[] for i in dname} 
        
        self.page = page

        # create a self dictionary to save all scraped information of properties
        pinfo = ['sold_time', 'address','price','property_type', 
             'area', 'beds', 'baths','parking','school','longterm_resident', 
                 'under_20', '20_39', '40_59','60+']
        self.pro_info = {i:[] for i in pinfo}
        
        
        
        # header for crawler
        self.header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
    
    def url_crawler(self):
        '''This function is to scrape all urls of properties of all districts'''
        
        for d in self.district_lis:
            # Start chrome service
            service = Service('C:\\Users\\yuwen\\chromedriver_win32\\chromedriver.exe') # change to the real path to your chromedriver
            service.start()

            # Define driver
            driver = webdriver.Remote(service.service_url)

            # Get webpages
            driver.get('http://www.domain.com.au/?mode=sold')

            main_url = 'http://www.domain.com.au'
            
            # Input position
            elem = driver.find_element_by_id("fe-pa-domain-home-typeahead-input")
            elem.clear()
            elem.send_keys(d)
        #     elem.send_keys(Keys.RETURN)
            elem.submit()


            for p in range(1,self.page+1):
                if p == 1:
                    # Get first page link
                    link = driver.current_url
                    # Use BS to Make a get request to fetch the webpage 
                    seek_page = requests.get(link, headers = self.header)
                    # Parsing the webpage using BeautifulSoup
                    seek_soup = bs(seek_page.text,"lxml")
                else:
                     # Use BS to Make a get request to fetch the webpage 
                    seek_page = requests.get(link, headers = self.header)
                    # Parsing the webpage using BeautifulSoup
                    seek_soup = bs(seek_page.text,"lxml")
                    
                     # link from second page
                    next_button = seek_soup.find('a', text=str(p))['href']
                    link = main_url + next_button

               

                # find all url of sold properties on p-th page
                temp_url = seek_soup.find_all('a', class_="address is-two-lines listing-result__address")
                for u in temp_url:
                    self.district_url[d].append(u["href"])
                                
                # signal for program excution check
                print(d, "page: {} done".format(p))
                
                # Pause the loop
                sleep(random.randint(1,5))
                
             # Write one district url into excel
            # Create a Pandas Excel writer 
            file_name = 'propertyurl.xlsx'
            writer = pd.ExcelWriter(file_name,engine=None,mode = 'a')

             # Write the dataframe with all information of one webpage to one sheet of an XlsxWriter Excel object.
            page_url = pd.DataFrame(self.district_url[d])
            page_url.to_excel(writer)
             # Close the Pandas Excel writer and output the Excel file.
            writer.save()
            
            driver.quit()
            
            return self.district_url

    
    def pro_info_crawler(self, durl=None):
        '''This function is to scrape all information of properties on scraped url'''
        if durl == None:
            durl = self.district_url
    
        for d in durl:
            count = 0
            for u in durl[d]:
                count+=1
                # signal for program excution check
                print(d, "{} url's properties information processing".format(count))
                
                # Use BS to Make a get request to fetch the webpage 
                page = requests.get(u, headers = self.header)
                # Parsing the webpage using BeautifulSoup
                soup = bs(page.text,"lxml")

                # Get information of the property
                sold_time = soup.find('span', {"class":"listing-details__listing-tag is-sold listing-details__summary-tag"}).get_text()
                self.pro_info['sold_time'].append(sold_time)

                address = soup.find('div', class_="listing-details__address-left-column").get_text()
                self.pro_info['address'].append(address.split('.')[0])

                price = soup.find('div', class_="listing-details__summary-title").get_text()
                self.pro_info['price'].append(price)


                temp1 = soup.find_all('div',class_="listing-details__key-features--value")
                try:
                    self.pro_info['property_type'].append(temp1[0].get_text())
                except:
                    self.pro_info['property_type'].append(None)
                               



                temp2 = soup.find_all('span', {'data-testid': "property-features-text-container"})
                try:
                    self.pro_info['beds'].append(temp2[0].get_text().split('.')[0])
                except:
                    self.pro_info['beds'].append(None)
                try:
                    self.pro_info['baths'].append(temp2[1].get_text())
                except:
                    self.pro_info['baths'].append(None)
                try:
                    self.pro_info['parking'].append(temp2[2].get_text())
                except:
                    self.pro_info['parking'].append(None)
                
                
                try:
                    self.pro_info['area'].append(temp1[1].get_text())
                except:
                    try:
                        self.pro_info['area'].append(temp[3].get_text())
                    except:
                        self.pro_info['area'].append(None)

                    
                s = soup.find('h5', class_="school-catchment__school-title").get_text()
                self.pro_info['school'].append(s)

                try:
                    ls = soup.find('div', class_="css-ibsnk8").get_text()
                    self.pro_info['longterm_resident'].append(ls)
                except:
                    self.pro_info['longterm_resident'].append(None)

                age = soup.find_all('td', class_="neighbourhood-insights__age-brackets-value")
                try:
                    self.pro_info['under_20'].append(age[0].get_text().split('}')[-1])
                except:
                    self.pro_info['under_20'].append(None)
                    
                try:
                    self.pro_info['20_39'].append(age[1].get_text().split('}')[-1])
                except:
                    self.pro_info['20_39'].append(None)
                    
                try:
                    self.pro_info['40_59'].append(age[2].get_text().split('}')[-1])
                except:
                    self.pro_info['40_59'].append(None)
                    
                try:
                    self.pro_info['60+'].append(age[3].get_text().split('}')[-1])
                except:
                    self.pro_info['60+'].append(None)
                
                
                # Pause the loop
                sleep(random.randint(1,5))
                

                

            # signal for program excution check
            print(d, "{} url's properties information done".format(count))
            
        # Write one district url into excel
        # Create a Pandas Excel writer 
        file_name = 'propertyinfo.xlsx'
        writer = pd.ExcelWriter(file_name,engine=None,mode = 'a')

         # Write the dataframe with all information of one webpage to one sheet of an XlsxWriter Excel object.
        pinfo = pd.DataFrame(self.pro_info)
        pinfo.to_excel(writer)
         # Close the Pandas Excel writer and output the Excel file.
        writer.save()
        
        return self.pro_info

In [10]:
dname = ['Epping', 'Carlingford', 'Dundas Valley', 'Telopea', 'Castle Hill', 'Honsby', 'Homebush']
page = 4

mycrawler = crawler(dname, page)

mycrawler.url_crawler()

Epping page: 1 done
Epping page: 2 done
Epping page: 3 done
Epping page: 4 done
Carlingford page: 1 done
Carlingford page: 2 done
Carlingford page: 3 done
Carlingford page: 4 done
Dundas Valley page: 1 done
Dundas Valley page: 2 done
Dundas Valley page: 3 done
Dundas Valley page: 4 done
Telopea page: 1 done
Telopea page: 2 done
Telopea page: 3 done
Telopea page: 4 done
Castle Hill page: 1 done
Castle Hill page: 2 done
Castle Hill page: 3 done
Castle Hill page: 4 done
Honsby page: 1 done
Honsby page: 2 done
Honsby page: 3 done
Honsby page: 4 done
Homebush page: 1 done
Homebush page: 2 done
Homebush page: 3 done
Homebush page: 4 done


In [9]:
url2={'Castle Hill':["https://www.domain.com.au/bg06-18-carlingford-road-epping-nsw-2121-2015717347"]}

In [11]:
name = ['Epping', 'Carlingford', 'Dundas Valley', 'Telopea', 'Castle Hill', 'Honsby', 'Homebush']
p = 4
testcrawler = crawler(name, p)

testcrawler.pro_info_crawler(url2)

Castle Hill 1 url's properties information processing
Castle Hill 1 url's properties information done


{'sold_time': ['Listing sold by advertiser 17th January 2020'],
 'address': ['BG06/18 Carlingford RoadEpping NSW 2121'],
 'price': ['SOLD - $535,000'],
 'property_type': ['Apartment / Unit / Flat'],
 'area': ['93m²'],
 'beds': ['1 '],
 'baths': ['1 Bath'],
 'parking': ['1 Parking'],
 'school': ['Epping Heights Public School'],
 'longterm_resident': ['50%'],
 'under_20': ['20%'],
 '20_39': ['38%'],
 '40_59': ['27%'],
 '60+': ['15%']}

In [14]:
dname = ['Turrmurra', 'Bella Vista']

page = 4

mycrawler2 = crawler(dname, page)

mycrawler2.url_crawler()

Turrmurra page: 1 done
Turrmurra page: 2 done
Turrmurra page: 3 done
Turrmurra page: 4 done


{'Turrmurra': ['https://www.domain.com.au/14-nagle-place-turramurra-nsw-2074-2016010961',
  'https://www.domain.com.au/21-2b-womerah-street-turramurra-nsw-2074-2015900184',
  'https://www.domain.com.au/18-pembroke-avenue-turramurra-nsw-2074-2015994267',
  'https://www.domain.com.au/116-1454-pacific-highway-turramurra-nsw-2074-2016023009',
  'https://www.domain.com.au/7-charlton-avenue-turramurra-nsw-2074-2016010978',
  'https://www.domain.com.au/32-trentino-road-turramurra-nsw-2074-2015876223',
  'https://www.domain.com.au/18-2-6-warrangi-street-turramurra-nsw-2074-2015879187',
  'https://www.domain.com.au/5-235-bobbin-head-road-turramurra-nsw-2074-2015827164',
  'https://www.domain.com.au/17-burnley-avenue-turramurra-nsw-2074-2015853315',
  'https://www.domain.com.au/86a-warragal-road-turramurra-nsw-2074-2015853384',
  'https://www.domain.com.au/40-geoffrey-street-turramurra-nsw-2074-2015854950',
  'https://www.domain.com.au/22-ashburton-avenue-turramurra-nsw-2074-2015880699',
  'http

In [15]:
mycrawler2.pro_info_crawler()

Turrmurra 1 url's properties information processing
Turrmurra 2 url's properties information processing
Turrmurra 3 url's properties information processing
Turrmurra 4 url's properties information processing
Turrmurra 5 url's properties information processing
Turrmurra 6 url's properties information processing
Turrmurra 7 url's properties information processing
Turrmurra 8 url's properties information processing
Turrmurra 9 url's properties information processing
Turrmurra 10 url's properties information processing
Turrmurra 11 url's properties information processing
Turrmurra 12 url's properties information processing
Turrmurra 13 url's properties information processing
Turrmurra 14 url's properties information processing
Turrmurra 15 url's properties information processing
Turrmurra 16 url's properties information processing
Turrmurra 17 url's properties information processing
Turrmurra 18 url's properties information processing
Turrmurra 19 url's properties information processing
Tu

{'sold_time': ['Listing sold by advertiser 1st February 2020',
  'Listing sold by advertiser 30th January 2020',
  'Listing sold by advertiser 24th January 2020',
  'Listing sold by advertiser 21st January 2020',
  'Listing sold by advertiser 21st January 2020',
  'Listing sold by advertiser 14th December 2019',
  'Listing sold by advertiser 6th December 2019',
  'Listing sold by advertiser 4th December 2019',
  'Listing sold by advertiser 30th November 2019',
  'Listing sold by advertiser 29th November 2019',
  'Listing sold by advertiser 28th November 2019',
  'Listing sold by advertiser 26th November 2019',
  'Listing sold by advertiser 26th November 2019',
  'Listing sold by advertiser 24th November 2019',
  'Listing sold by advertiser 23rd November 2019',
  'Listing sold by advertiser 22nd November 2019',
  'Listing sold by advertiser 21st November 2019',
  'Listing sold by advertiser 21st November 2019',
  'Listing sold by advertiser 20th November 2019',
  'Listing sold by advert